In [11]:
from patterns.abstract import *
from patterns.entailment import *
from patterns.contradiction import *
from patterns.neutral import *
from functools import partial
import pandas as pd

In [6]:
se1 = StructuredExplanation('∧', [StructuredExplanation('⊆', ['cat', 'animal']), StructuredExplanation('→', ['car', 'vehicle'])])
se2 = StructuredExplanation('∧', [StructuredExplanation('→', ['car', 'vehicle']), StructuredExplanation('⊆', ['cat', 'animal'])])

print(se1 == se2)

True


In [8]:
eval_data = pd.read_csv("data/cleaned_assigned_samples_training.csv")

In [10]:
patterns = [RephrasingPattern(), ImplicationPattern(), EquivalencePattern(), IfThenPattern(), ClassificationPattern()]
get_highlights = lambda n, r: ast.literal_eval(r[f'Sentence1_Highlighted_Ordered_{n}']) + ast.literal_eval(r[f'Sentence2_Highlighted_Ordered_{n}'])
apply_patterns = lambda n, x: [pattern(nlp(x[f'Explanation_{n}']), get_highlights(n, x)) for pattern in patterns]
concat = lambda x: AbstractPattern.concatenate_explanations(x)

In [18]:

ent_results = eval_data[eval_data["gold_label"] == "entailment"].copy()

ent_results['Explanation_1_Result'] = ent_results.apply(partial(apply_patterns, 1), axis=1).apply(concat)

ent_output = ent_results[[
    'Explanation_1', 'Explanation_1_Result', 'structured_explanation'
]]

ent_output.reset_index(drop=True, inplace=True)

display(ent_output)

KeyError: "['structured_explanation'] not in index"

In [13]:
patterns = [NotRephrasingPattern(), NotImplicationPattern(), NotEquivalencePattern(), XORPattern(), IfThenPattern(), NotClassificationPattern()]
get_highlights = lambda n, r: ast.literal_eval(r[f'Sentence1_Highlighted_Ordered_{n}']) + ast.literal_eval(r[f'Sentence2_Highlighted_Ordered_{n}'])
apply_patterns = lambda n, x: [pattern(nlp(x[f'Explanation_{n}']), get_highlights(n, x)) for pattern in patterns]
concat = lambda x: AbstractPattern.concatenate_explanations(x)

In [14]:

con_results = eval_data[eval_data["gold_label"] == "contradiction"].copy()

con_results['Explanation_1_Result'] = con_results.apply(partial(apply_patterns, 1), axis=1)

con_output = con_results[[
    'Sentence1', 'Sentence2',
    'Sentence1_Highlighted_Ordered_1', 'Sentence2_Highlighted_Ordered_1',
    'Explanation_1', 'Explanation_1_Result',
]]

con_output.reset_index(drop=True, inplace=True)